In [3]:
from ase import Atoms
from gpaw import GPAW

d = 0.74
a = 6.0

atoms = Atoms('H2',
              positions=[[0, 0, 0],
                         [0, 0, d]],
              cell=[a, a, a])

atoms.center()

calc = GPAW(mode='fd', nbands=2)
atoms.calc = calc
print(atoms.get_forces())


  ___ ___ ___ _ _ _  
 |   |   |_  | | | | 
 | | | | | . | | | | 
 |__ |  _|___|_____|  24.7.0b1
 |___|_|             

User:   rakshit@gigavolt
Date:   Wed Dec 18 18:00:34 2024
Arch:   x86_64
Pid:    14034
CWD:    /home/rakshit/deepchem-1/examples/tutorials
Python: 3.11.10
gpaw:   /home/rakshit/anaconda3/envs/deepchem-cpu/lib/python3.11/site-packages/gpaw
_gpaw:  /home/rakshit/anaconda3/envs/deepchem-cpu/lib/python3.11/site-packages/
        _gpaw.cpython-311-x86_64-linux-gnu.so (cbb5a734e9)
ase:    /home/rakshit/anaconda3/envs/deepchem-cpu/lib/python3.11/site-packages/ase (version 3.23.0)
numpy:  /home/rakshit/anaconda3/envs/deepchem-cpu/lib/python3.11/site-packages/numpy (version 2.0.1)
scipy:  /home/rakshit/anaconda3/envs/deepchem-cpu/lib/python3.11/site-packages/scipy (version 1.14.1)
libxc:  7.0.0
units:  Angstrom and eV
cores: 1
OpenMP: False
OMP_NUM_THREADS: 1

Input parameters:
  mode: fd
  nbands: 2

System changes: positions, numbers, cell, pbc, initial_charges, initial_mag

In [1]:
# Atomization Energy (N2)

from ase import Atoms
from ase.calculators.emt import EMT

atom = Atoms('N')
atom.calc = EMT()
e_atom = atom.get_potential_energy()

d = 1.1
molecule = Atoms('2N', [[0., 0., 0.], [0., 0., d]])
molecule.calc = EMT()
e_molecule = molecule.get_potential_energy()

e_atomization = e_molecule - 2 * e_atom

print(e_atom)
print(e_molecule)
print(-e_atomization)

5.1
0.440343573035614
9.759656426964385


In [3]:
# Atomization Energy (H20)

from ase import Atoms
from ase.calculators.emt import EMT

H_atom = Atoms('H')
H_atom.calc = EMT()
H_e_atom = atom.get_potential_energy()

H_atom = Atoms('H')
H_atom.calc = EMT()
H_e_atom = atom.get_potential_energy()

d = 1.1
molecule = Atoms('2N', [[0., 0., 0.], [0., 0., d]])
molecule.calc = EMT()
e_molecule = molecule.get_potential_energy()

e_atomization = e_molecule - 2 * e_atom

print(e_atom)
print(e_molecule)
print(-e_atomization)

5.1
0.440343573035614
9.759656426964385
